## MLflow's Model Registry

In [1]:
from mlflow.tracking import MlflowClient


MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

### Interacting with the MLflow tracking server

The `MlflowClient` object allows us to interact with...
- an MLflow Tracking Server that creates and manages experiments and runs.
- an MLflow Registry Server that creates and manages registered models and model versions. 

To instantiate it we need to pass a tracking URI and/or a registry URI

In [2]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

client.list_experiments()

[<Experiment: artifact_location='./mlruns/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='my-cool-experiment', tags={}>,
 <Experiment: artifact_location='./mlruns/2', experiment_id='2', lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='./mlruns/3', experiment_id='3', lifecycle_stage='active', name='my-experiment-1', tags={}>]

In [4]:
client.create_experiment(name="my-test-experiment")

'4'

Let's check the latest versions for the experiment with id `1`...

In [31]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='2', # nyc_taxi_experiment
    filter_string="metrics.rmse < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [82]:
runs[0].info.run_id

'f9134e3e3b2940aabbd650c52193b685'

In [32]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: f9134e3e3b2940aabbd650c52193b685, rmse: 6.3040
run id: 188043da0d874241ab2d163b9e6db689, rmse: 6.3040
run id: 1135a36902a34c28b25b2522b1ce8b93, rmse: 6.4164
run id: df7072fb57a6458e98973ab52b972851, rmse: 6.7423
run id: c4bb9c5262c240e893e8406e78a2ea05, rmse: 6.9118


In [84]:
runs[0].data.params

{'learning_rate': '0.09585355369315604',
 'max_depth': '30',
 'min_child_weight': '1.060597050922164',
 'objective': 'reg:linear',
 'reg_alpha': '0.018060244040060163',
 'reg_lambda': '0.011658731377413597',
 'seed': '42'}

### Interacting with the Model Registry

In this section We will use the `MlflowClient` instance to:

1. Register a new version for the experiment `nyc-taxi-regressor`
2. Retrieve the latests versions of the model `nyc-taxi-regressor` and check that a new version `4` was created.
3. Transition the version `4` to "Staging" and adding annotations to it.

In [8]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [16]:
run_id = "188043da0d874241ab2d163b9e6db689"
model_uri = f"runs:/{run_id}/model"

In [34]:
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
2022/05/30 17:32:12 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: nyc-taxi-regressor, version 3
Created version '3' of model 'nyc-taxi-regressor'.


<ModelVersion: creation_timestamp=1653953532380, current_stage='None', description=None, last_updated_timestamp=1653953532380, name='nyc-taxi-regressor', run_id='188043da0d874241ab2d163b9e6db689', run_link=None, source='./mlruns/2/188043da0d874241ab2d163b9e6db689/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [35]:
model_uri

'runs:/188043da0d874241ab2d163b9e6db689/model'

## Transition a model

In [63]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 2, stage: Staging
version: 3, stage: Production


In [53]:
model_version = 3
new_stage = "Production"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

<ModelVersion: creation_timestamp=1653953532380, current_stage='Production', description=None, last_updated_timestamp=1653954692117, name='nyc-taxi-regressor', run_id='188043da0d874241ab2d163b9e6db689', run_link=None, source='./mlruns/2/188043da0d874241ab2d163b9e6db689/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [64]:
from datetime import datetime
model_version = 3
date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: creation_timestamp=1653953532380, current_stage='Production', description='The model version 3 was transitioned to Production on 2022-05-30', last_updated_timestamp=1653955046304, name='nyc-taxi-regressor', run_id='188043da0d874241ab2d163b9e6db689', run_link=None, source='./mlruns/2/188043da0d874241ab2d163b9e6db689/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

### Comparing versions and selecting the new "Production" model

In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

1. Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
2. Download the `DictVectorizer` that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
3. Preprocess the test set using the `DictVectorizer` so we can properly feed the regressors.
4. Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
5. Based on the results, update the "Production" model version accordingly.


**Note: the model registry doesn't actually deploy the model to production when you transition a model to the "Production" stage, it just assign a label to that model version. You should complement the registry with some CI/CD code that does the actual deployment.**

In [65]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [66]:
df = read_dataframe("data/green_tripdata_2021-03.parquet")

In [67]:
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

'/home/ovokpus/mlops-learn/02-experiment-tracking/preprocessor'

In [68]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [69]:
X_test = preprocess(df, dv)

In [70]:
target = "duration"
y_test = df[target].values

In [72]:
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

OSError: No such file or directory: './mlruns/2/188043da0d874241ab2d163b9e6db689/artifacts/model/.'

In [73]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

CPU times: user 122 ms, sys: 0 ns, total: 122 ms
Wall time: 119 ms


{'rmse': 6.659623830022513}

In [ ]:
client.transition_model_version_stage(
    name=model_name,
    version=4,
    stage="Production",
    archive_existing_versions=True
)